In [1]:
from transformers import AutoTokenizer

import sys

# sys.path.append(os.path.dirname(os.path.abspath(__file__)))

sys.path.append("..") # relative import issue

from processor import QAProcessor
from arguments import DatasetArguments

In [2]:
from datasets import Dataset, DatasetBuilder
import pandas as pd

In [3]:
wiki_dir = "/opt/ml/data/wikipedia_documents.json"
wiki_df = pd.read_json(wiki_dir).T
wiki_dataset = Dataset.from_pandas(wiki_df)

In [4]:
wiki_dataset.features

{'text': Value(dtype='string', id=None),
 'corpus_source': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None),
 'domain': Value(dtype='null', id=None),
 'title': Value(dtype='string', id=None),
 'author': Value(dtype='null', id=None),
 'html': Value(dtype='null', id=None),
 'document_id': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [5]:
subset = wiki_dataset.select(range(16))

In [6]:
MAX_CHUNK_LEN = 500
DOC_STRIDE = 100

from collections import defaultdict

def split_into_chunks(examples):

    outputs = defaultdict(list)

    column_names = list(examples.keys())
    column_names.remove("text")

    texts = examples['text']

    for example_idx, text in enumerate(texts):
        split_ids = list(range(0, len(text), MAX_CHUNK_LEN-DOC_STRIDE)) + [len(text)]

        num_chunks = 0
        for i in range(len(split_ids)-1):
            start_idx, end_idx = split_ids[i], min(split_ids[i+1]+DOC_STRIDE, len(text))
            if end_idx - start_idx < DOC_STRIDE:
                # already fully included in the previous chunk!
                # therefore, safe to break out the loop!
                break

            split_text = text[start_idx:end_idx]
            split_text = split_text.strip()
            outputs['text'].append(split_text)
            num_chunks += 1
        
        for col in column_names:
            outputs[col].extend([examples[col][example_idx]] * num_chunks)

    return outputs

In [ ]:
chunked_subset = subset.map(split_into_chunks, batched=True, num_proc=4)

In [26]:
def add_title_to_text(example):
        
    prefix = example['title'] + ": "

    example['text'] = prefix + example['text']
    # example['start_position'] += len(prefix) # wiki itself doesnt' contain start_position, end_position
    # example['end_position'] += len(prefix)

    return example

In [27]:
def split_into_sentences(examples):

    outputs = defaultdict(list)

    column_names = list(examples.keys())
    column_names.remove("text")

    texts = examples['text']

    for example_idx, text in enumerate(texts):
        subtexts = [subtext for subtext in text.split("\n") if len(subtext) > 0]

        current_context = ""
        num_chunks = 0

        for subtext in subtexts:
            if len(current_context) + len(subtext) < MAX_CHUNK_LEN:
                current_context += subtext
            else:
                outputs['text'].append(current_context)
                num_chunks += 1
                current_context = subtext

        # extreme case
        if ("text" not in outputs) or (current_context != outputs['text'][-1]):
            outputs['text'].append(current_context)
            num_chunks += 1

        for col in column_names:
            outputs[col].extend([examples[col][example_idx]] * num_chunks)
    
    return outputs

In [28]:
new_chunked_subset = subset.map(split_into_sentences, batched=True, num_proc=1)

In [29]:
titled_chunked_subset = new_chunked_subset.map(add_title_to_text)

In [32]:
titled_chunked_subset['text']

['나라 목록: 이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.두 목록은 모두 가나다 순이다.일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 '나라 목록: 이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정에 따르면, 국가는 다음의 조건을 만족해야 한다.* (a) 영속적인 국민* (b) 일정한 영토* (c) 정부* (d) 타국과의 관계 참여 자격.특히, 마지막 조건은 국제 공동체의 참여 용인을 내포하고 있기 때문에, 다른 나라의 승인이 매우 중요한 역할을 할 수 있다.  이 목록에 포함된 모든 국가는 보통 이 기준을 만족하는 것으로 보이는 자주적이고 독립적인 국가이다. 하지만 몬테비데오 협약 기준을 만족하는지의 여부는 많은 국가가 논쟁이 되고 있는 실정이다. 또한, 몬테비데오 협약 기준만이 국가 지위의 충분한 자격이든 아니든, 국제법의 견해 차이는 존재할 수 있다. 이 물음에 대한 다른 이론에 대한 고리는 아래에서 볼 수 있다.위 기준에 논거하여 이 목록은 다음 206개 국가를 포함하고 있다.* 일반 국제 승인을 받은 195개 자주 국가.** 유엔 가입 국가 193개',
 '나라 목록: ** 성좌의 명칭으로 유엔에서 국제 승인을 받은 국가: 바티칸 시국** 팔레스

In [2]:
MODEL_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [6]:
dataset_args = DatasetArguments()

In [7]:
qa_processor = QAProcessor(dataset_args, tokenizer)

In [8]:
train_examples = qa_processor.get_train_examples()

In [9]:
train_examples.features

{'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
  'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)},
 'document_id': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64', id=None)}

In [16]:
def _is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False

In [21]:
subsets = train_examples.select(range(16))

In [37]:
import re

punct_mapping = {
    "‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", 
    "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 
    'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean_texts(texts): 
    cleaned_texts = []
    for text in texts: 

        for subtext in text.splitlines():

            text = re.sub('\n', ' ', str(text)) # remove newline
            text = re.sub('\t', ' ', text) # remove newline

            text = re.sub(r'[@%\\*=/~#&\+á?\xc3\xa1\-\|\:\;\-\_\$]', '.', text) #remove punctuation 
            # review = re.sub(r'\d+','', str(texts[i])) # remove number 
            # review = review.lower() #lower case 
            
            text = re.sub(r'\s+', ' ', text) #remove extra space 
            text = re.sub(r'<[^>]+>','',text) #remove Html tags 
            text = re.sub(r"^\s+", '', text) #remove space from start 
            text = re.sub(r'\s+$', '', text) #remove space from the end 
        
        cleaned_texts.append(text) 

    return cleaned_texts

# https://ebbnflow.tistory.com/246

In [49]:
text = subsets["context"][0]

In [50]:
[text for text in text.split("\\n") if len(text) > 0]

['미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.',
 '미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.',
 '미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05']

In [63]:
MAX_LENGTH = 1000
new_texts = []

subtexts = [t for t in text.split("\\n") if len(t) > 0]

combined = ""
for subtext in subtexts:
    if len(combined) + len(subtext) < MAX_LENGTH:
        combined += " " + subtext
        print(len(combined) + len(subtext), combined)
    else:
        new_texts.append(combined.strip())
        combined = subtext
if new_texts[-1] != combined:
    new_texts.append(combined)

113  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.
276  미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.


In [64]:
new_texts

['미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다. 미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.',
 '미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05']

In [65]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/roberta-base")

In [70]:
tokenized_sentence = tokenizer(new_texts, truncation=True)
print([len(input) for input in tokenized_sentence['input_ids']])
print([len(t) for t in new_texts])

[86, 342]
[166, 689]


In [40]:
contexts

['미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 "'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 1950년대이다. 2차 세계대전을 마치고, 6.25전쟁의 시기로 유럽은 전후 재건에 집중하고, 유럽 제국주의의 식민지가 독립하여 아프리카, 아시아, 

In [18]:
ids = list(range(0, len(new_context), 64)) + [len(new_context)]

sub_contexts = []

for i in range(len(ids)-1):
    sub_context = new_context[ids[i]:ids[i+1]]
    sub_contexts.append(sub_context)

In [19]:
sub_contexts

['미국상의원또는미국상원(UnitedStatesSenate)은양원제인미국의회의상원이다.\\n\\n미국부통령이상원의장이된다.',
 '각주당2명의상원의원이선출되어100명의상원의원으로구성되어있다.임기는6년이며,2년마다50개주중1/3씩상원의원을새로선출하',
 '여연방에보낸다.\\n\\n미국상원은미국하원과는다르게미국대통령을수반으로하는미국연방행정부에각종동의를하는기관이다.하원이세금과',
 '경제에대한권한,대통령을포함한대다수의공무원을파면할권한을갖고있는국민을대표하는기관인반면상원은미국의주를대표한다.즉캘리포니아',
 '주,일리노이주같이주정부와주의회를대표하는기관이다.그로인하여군대의파병,관료의임명에대한동의,외국조약에대한승인등신속을요하는',
 '권한은모두상원에게만있다.그리고하원에대한견제역할(하원의법안을거부할권한등)을담당한다.2년의임기로인하여급진적일수밖에없는하',
 '원은지나치게급진적인법안을만들기쉽다.대표적인예로건강보험개혁당시하원이미국연방행정부에게퍼블릭옵션(공공건강보험기관)의조항이',
 '있는반면상원의경우하원안이지나치게세금이많이든다는이유로퍼블릭옵션조항을제외하고비영리건강보험기관이나보험회사가담당하도록한것이',
 '다.이경우처럼상원은하원이나내각책임제가빠지기쉬운국가들의국회처럼걸핏하면발생하는의회의비정상적인사태를방지하는기관이다.상원은',
 '급박한처리사항의경우가아니면법안을먼저내는경우가드물고하원이만든법안을수정하여다시하원에되돌려보낸다.이러한방식으로단원제가빠지',
 '기쉬운함정을미리방지하는것이다.날짜=2017-02-05']